In [9]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import os
import json
import matplotlib

matplotlib.use("pdf")

In [10]:
df = pd.DataFrame(columns=['Dataset','Regularization','Percentage', 'Model','Test/Accuracy','Test/Loss'])
for dataset in os.listdir("../results/"):
    for experiment in os.listdir(f"../results/{dataset}/"):
        event_acc = EventAccumulator(f"../results/{dataset}/{experiment}/")
        config_file = json.load(open(f"../results/{dataset}/{experiment}/config.json"))
        event_acc.Reload()
        
        Test_Acc = None
        Test_Loss = None
        regularization = ""
        
        if "Test/Accuracy" in event_acc.Tags()["scalars"]:
            Test_Acc = event_acc.Scalars("Test/Accuracy")[-1].value
        
        if "Test/Loss" in event_acc.Tags()["scalars"]:
            Test_Loss = event_acc.Scalars("Test/Loss")[-1].value
        
        if config_file["xshield"]:
            regularization = "X-SHIELD"
        if config_file["shield"]:
            regularization = "SHIELD"
        if regularization == "":
            regularization = "Baseline"
        df = pd.concat([df, pd.DataFrame({"Dataset": dataset, 
                                        "Regularization": regularization,
                                        "Model":config_file["pretrained_model"],
                                        "Test/Accuracy": Test_Acc,
                                        "Test/Loss": Test_Loss,
                                        "Percentage": config_file["percentage"]}, index=[0])], ignore_index=True)

/tmp/ipykernel_184243/2651833224.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({"Dataset": dataset,
/tmp/ipykernel_184243/2651833224.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({"Dataset": dataset,
/tmp/ipykernel_184243/2651833224.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when det

In [11]:
df_latex = df.copy()
df_latex.rename(
    columns={
        "Dataset": "Dataset",
        "Percentage": "Percentage",
        "Model": "Model",
        "Test/Accuracy": "Accuracy",
        "Test/Loss": "Loss",
    },
    inplace=True,
)

df_latex = df_latex[df_latex["Regularization"].isin(["Baseline", "X-SHIELD", "SHIELD"])]
df_latex = df_latex[df_latex["Model"].isin(["efficientnet-b2", "efficientnet_v2_s"])]
df_latex["Accuracy"] = df_latex["Accuracy"].round(4)
df_latex["Loss"] = df_latex["Loss"].round(4)
df_latex = df_latex.sort_values(by=["Dataset", "Model", "Percentage", "Regularization"])

df_latex["Accuracy"] = (
    (df_latex["Accuracy"] * 100)
    .round(2)
    .map(lambda x: f"{x}\%")
    .map(lambda x: x.replace(".", ","))
)
df_latex["Loss"] = (
    df_latex["Loss"].round(4).map(lambda x: f"{x}").map(lambda x: x.replace(".", ","))
)

df_latex[["Accuracy", "Loss"]].to_csv("results.csv", index=False, sep=";")

df_latex["Percentage"] = df_latex["Percentage"].map(lambda x: f"{int(x)}\%")
df_latex = df_latex[
    ["Dataset", "Model", "Percentage", "Regularization", "Accuracy", "Loss"]
]
df_latex["Model"] = df_latex["Model"].map(lambda x: x.replace("_", " "))
df_latex["Model"] = df_latex["Model"].map(lambda x: x.replace("-", " "))

for dataset in df_latex["Dataset"].unique():
    df_ds = df_latex[df_latex["Dataset"] == dataset].drop(columns=["Dataset"])
    # df_ds = df_ds.drop(columns=["Model"])
    # Ordenar df_ds primero por modelo y luego por porcentaje

    df_ds.to_latex(
        f"tables/{dataset}_X-SHIELD.tex",
        index=False,
        escape=False,
        column_format="llccc",
    )